In [ ]:
import yaml 
from nupic.algorithms.spatial_pooler import SpatialPooler
from nupic.algorithms.temporal_memory import TemporalMemory
from nupic.encoders.multi import  MultiEncoder
from  nupic.encoders.scalar import ScalarEncoder
from nupic.encoders.category import CategoryEncoder
from nupic.algorithms.sdr_classifier import  SDRClassifier
import csv 
from time import time
import ipywidgets as wg
from IPython.display import display
import numpy


In [ ]:
        PARAMS_PATH =  ("model_iot.yaml")
        with open(PARAMS_PATH, "r") as f:
            modelParams = yaml.safe_load(f)["modelParams"]   
            spParams = modelParams["spParams"]
            tmParams = modelParams["tmParams"]
            clParams = modelParams["clParams"]
        sink_tap_Encoder = ScalarEncoder(name="sink_tap", w=7, n=14, minval=0, maxval=1,forced=True)
        toilet_Encoder = ScalarEncoder(name="toilet", w=7, n=14, minval=0, maxval=1,forced=True)
        shower_Encoder = ScalarEncoder(name="shower", w=7, n=14, minval=0, maxval=1,forced=True)

        baselineEncoder = ScalarEncoder(name = "baseline",w = 21, n = 2625,minval= 47,maxval=75,forced= True)

        flowEncoder = ScalarEncoder(name="flow", w=15, n=1350, minval=0, maxval=11,forced = True)
        encodingWidth = (sink_tap_Encoder.getWidth()+flowEncoder.getWidth()+baselineEncoder.getWidth()
                                +toilet_Encoder.getWidth()+flowEncoder.getWidth()+baselineEncoder.getWidth()
                                +shower_Encoder.getWidth()+flowEncoder.getWidth()+baselineEncoder.getWidth())
        sp = SpatialPooler(
            inputDimensions=(encodingWidth,),
            columnDimensions=(spParams["columnCount"],),
            potentialPct=spParams["potentialPct"],
            potentialRadius=encodingWidth,
            globalInhibition=spParams["globalInhibition"],
            localAreaDensity=spParams["localAreaDensity"],
            numActiveColumnsPerInhArea=spParams["numActiveColumnsPerInhArea"],
            synPermInactiveDec=spParams["synPermInactiveDec"],
            synPermActiveInc=spParams["synPermActiveInc"],
            synPermConnected=spParams["synPermConnected"],
            boostStrength=spParams["boostStrength"],
            seed=spParams["seed"],
            wrapAround=True
        )
        tm = TemporalMemory(
            columnDimensions=(tmParams["columnCount"],),
            cellsPerColumn=tmParams["cellsPerColumn"],
            activationThreshold=tmParams["activationThreshold"],
            initialPermanence=tmParams["initialPerm"],
            connectedPermanence=spParams["synPermConnected"],
            minThreshold=tmParams["minThreshold"],
            maxNewSynapseCount=tmParams["newSynapseCount"],
            permanenceIncrement=tmParams["permanenceInc"],
            permanenceDecrement=tmParams["permanenceDec"],
            predictedSegmentDecrement=tmParams["predictedSegmentDecrement"],
            maxSegmentsPerCell=tmParams["maxSegmentsPerCell"],
            maxSynapsesPerSegment=tmParams["maxSynapsesPerSegment"],
            seed=tmParams["seed"]
        )
        classifier = SDRClassifier(
            steps  = [1],alpha=clParams["alpha"],verbosity= clParams["verbosity"]
        )
        classifier1 = SDRClassifier(
            steps  = [1],alpha=clParams["alpha"],verbosity= clParams["verbosity"]
        )
        classifier2 = SDRClassifier(
            steps  = [1],alpha=clParams["alpha"],verbosity= clParams["verbosity"]
        )


In [ ]:
     def runLearning(numRecords):
        classifier_counter = 0
        learning_time = time()
        with open("test.csv", "r") as fin:
            reader = csv.reader(fin)
            headers = reader.next()
            reader.next()
            reader.next()
            progress = wg.IntProgress(min = 0,max = 1000,value = 0,description = "Loading",step = 1,bar_style='info')
            number_iterations_widget = wg.IntSlider(min = 1000,max = 20000,value = 0,step =500,descroption="Number of iterations",orientation='horizontal')
            
            display(progress,number_iterations_widget)
            for count, record in enumerate(reader):
                #print "Count",count
                progress.value = count
                
                if count >= numRecords: break

                # Convert data value string into float.
                sink_tap_value = float(record[2]) # device 1
                toilet_value = float(record[3]) #device 2
                shower__value = float(record[4]) # device 3

                 # event_value_7 = float(record[8]) # device 7
                baseline_all = float(record[6])
                flow_value  = float(record[0])
                # To encode, we need to provide zero-filled numpy arrays for the encoders
                # to populate.
                eventBits = numpy.zeros(sink_tap_Encoder.getWidth())
                eventBits_2 = numpy.zeros(toilet_Encoder.getWidth())
                eventBits_3 = numpy.zeros(shower_Encoder.getWidth())

                baseline_Bits = numpy.zeros(baselineEncoder.getWidth())
                flowBits = numpy.zeros(flowEncoder.getWidth())

                 # Now we call the encoders to create bit representations for each value.
                sink_tap_Encoder.encodeIntoArray(sink_tap_value, eventBits)
                toilet_Encoder.encodeIntoArray(toilet_value,eventBits_3)
                shower_Encoder.encodeIntoArray(shower__value,eventBits_2) 

                baselineEncoder.encodeIntoArray(baseline_all,baseline_Bits)
                flowEncoder.encodeIntoArray(flow_value, flowBits)

                # Concatenate all these encodings into one large encoding for Spatial
                 # Pooling.
                encoding = numpy.concatenate(
                    [eventBits,flowBits,baseline_Bits,eventBits_2,flowBits,baseline_Bits,eventBits_3,flowBits,baseline_Bits]
                )

                # Create an array to represent active columns, all initially zero. This
                 # will be populated by the compute method below. It must have the same
                 # dimensions as the Spatial Pooler.
                activeColumns = numpy.zeros(spParams["columnCount"])
                 # activeColumns1 = numpy.zeros(spParams["columnCount"])

                 # Execute Spatial Pooling algorithm over input space.

                sp.compute(encoding,True,activeColumns)

                # sp.compute(encoding1, True, activeColumns)

                activeColumnIndices = numpy.nonzero(activeColumns)[0]

                 # Execute Temporal Memory algorithm over active mini-columns.
                tm.compute(activeColumnIndices, learn=True)

                activeCells = tm.getActiveCells()

                # Get the bucket info for this input value for classification.
                bucketIdx_sink_tap  =  sink_tap_Encoder.getBucketIndices(sink_tap_value)[0] 
                bucketIdx_toilet = toilet_Encoder.getBucketIndices(toilet_value)[0]
                bucketIdx_shower = shower_Encoder.getBucketIndices(shower__value)[0]

                # Run classifier to translate active cells back to scalar value.
                classifierResult = classifier.compute(
                    recordNum=classifier_counter,
                    patternNZ=activeCells,
                    classification={
                        "bucketIdx": bucketIdx_sink_tap,
                        "actValue": sink_tap_value
                    },
                    learn=True,
                    infer=False
                )
                classifierResult1 = classifier1.compute(
                    recordNum=classifier_counter,
                    patternNZ=activeCells,
                    classification={
                        "bucketIdx": bucketIdx_toilet,
                        "actValue": toilet_value
                    },
                    learn=True,
                    infer=False
                )

                classifierResult2 = classifier2.compute(
                    recordNum=classifier_counter,
                    patternNZ=activeCells,
                    classification={
                        "bucketIdx": bucketIdx_shower,
                        "actValue": shower__value
                    },
                    learn=True,
                    infer=False
                )
                classifier_counter += 1
                learning_time_end = time()

           #     print "Time",(learning_time - learning_time_end)

            


In [ ]:
runLearning(1000)